In [12]:
from math import *
from tqdm import tqdm
import numpy as np
from PIL import Image

In [55]:
class sphere():
    def __init__(self, ce, r, co, spe, refl, fuz, spht = 'normal'):
        self.spheretype = spht
        self.center = ce
        self.radius = r
        self.color = co
        self.specular = spe
        self.reflective = refl
        self.fuzzy = fuz

spheres = []

spheres.append(sphere(np.array([0, 0, 5]), 1, np.array([238, 130, 238]),  10, 0.3, 0, 'main'))#pink
spheres.append(sphere(np.array([0, -0.7, 4.3]), 0.45, np.array([0, 255, 255]),  10, 0.3, 0))#blue
spheres.append(sphere(np.array([0.7, 0.4, 4.3]), 0.45, np.array([0, 255, 255]),  10, 0.3, 0))#blue
spheres.append(sphere(np.array([-0.6, 0.4, 4.3]), 0.45, np.array([0, 255, 255]),  10, 0.3, 0))#blue
spheres.append(sphere(np.array([-2, -5, 5]), 4, np.array([80, 200, 120]), 25, 0.3, 0, 'main'))#green
spheres.append(sphere(np.array([1.5, 1.5, 4.2]), 0.16, np.array([255, 255, 255]), 0, 0, 0,'spherelight'))#light

def IntersectRaySphere(O, d, sphere):
    center = sphere.center
    radius = sphere.radius
    oc = O - center

    k1 = np.sum(np.dot(d, d))
    k2 = 2*np.sum(np.dot(oc, d))
    k3 = np.sum(np.dot(oc, oc)) - radius*radius

    discriminant = k2*k2 - 4*k1*k3
    if discriminant < 0:
        return inf, inf

    t1 = (-k2 + sqrt(discriminant)) / (2*k1)
    t2 = (-k2 - sqrt(discriminant)) / (2*k1)
    return t1, t2

def ClosestIntersection(O, d, tmin, tmax):
    closest_t = inf
    closest_sphere = None
  
    intersectpoints = []
    mainpoints = []
    for sphere in spheres:
        t1, t2 = IntersectRaySphere(O, d, sphere)
        intersectpoints.append(np.array([t1, t2]))
        if tmin <= t1 < tmax:
            mainpoints.append(t1)
        if tmin <= t1 < tmax:
            mainpoints.append(t2)
               
    intersectpoints = np.array(intersectpoints)
    mainpoints = np.array(mainpoints)
    mainpoints = np.sort(mainpoints)
    mainpoints = np.trim_zeros(mainpoints)
    
    for countip, ip in enumerate(intersectpoints):
        if (mainpoints.size > 0 and (mainpoints[0] == ip[0] or mainpoints[0] == ip[1]) 
            and ((countip == 0 and not(intersectpoints[1][0] < mainpoints[0] < intersectpoints[1][1]) 
                 and not(intersectpoints[2][0] < mainpoints[0] < intersectpoints[2][1])
                 and not(intersectpoints[3][0] < mainpoints[0] < intersectpoints[3][1]))
                 or countip == 4 or countip == 5)):
            closest_t = mainpoints[0]
            closest_sphere = spheres[countip]
            break
        elif (mainpoints.size > 0 and (mainpoints[0] == ip[0] or mainpoints[0] == ip[1]) 
              and countip != 0 and (tmin <= intersectpoints[0][0] < tmax or tmin <= intersectpoints[0][1] < tmax)):
            if mainpoints[0] == ip[0]:
                closest_t = ip[1]
            else:
                closest_t = ip[0]   
            closest_sphere = spheres[countip]
            break
        elif (mainpoints.size > 0 and (mainpoints[0] == ip[0] or mainpoints[0] == ip[1]) 
              and countip != 0 and (intersectpoints[0][0] == inf or intersectpoints[0][1] == inf or
                                   intersectpoints[0][0] < tmin or intersectpoints[0][1] < tmin)):
            mainpoints = np.delete(mainpoints, np.where(mainpoints == ip[0])[0][0])
            mainpoints = np.delete(mainpoints, np.where(mainpoints == ip[1])[0][0])
            """
            if (intersectpoints[5][0] != 0 or intersectpoints[5][1] != 0):
                
                if (intersectpoints[5][0] <= intersectpoints[5][1] and 
                    intersectpoints[5][0] >= tmin and intersectpoints[5][0] <= tmax):
                    closest_t = intersectpoints[5][0]
                    closest_sphere = spheres[5]
                elif (intersectpoints[5][1] < intersectpoints[5][0] and 
                    intersectpoints[5][1] >= tmin and intersectpoints[5][1] <= tmax):
                    closest_t = intersectpoints[5][1]
                    closest_sphere = spheres[5]
            elif (intersectpoints[4][0] != 0 or intersectpoints[4][1] != 0):
                if (intersectpoints[4][0] <= intersectpoints[4][1] and 
                    intersectpoints[4][0] >= tmin and intersectpoints[4][0] <= tmax):
                    closest_t = intersectpoints[4][0]
                    closest_sphere = spheres[4]
                elif (intersectpoints[4][1] < intersectpoints[4][0] and 
                    intersectpoints[4][1] >= tmin and intersectpoints[4][1] <= tmax):
                    closest_t = intersectpoints[4][1]
                    closest_sphere = spheres[4]
         """
                    
        
    return closest_sphere, closest_t

def TraceRay(O, d, tmin, tmax, depth):
    closest_sphere, closest_t = ClosestIntersection(O, d, tmin, tmax)
        
    if closest_sphere == None:
        return BACKGROUND_COLOR
    
    P = O + closest_t * d  # вычисление пересечения
    if closest_sphere.spheretype == 'main':
        N = P - closest_sphere.center # вычисление нормали сферы в точке пересечения
    else:
        N = closest_sphere.center - P
    N = N / sqrt(np.sum(np.square(N)))
    
    if closest_sphere.spheretype == 'spherelight':
        return closest_sphere.color 
    
    local_color = closest_sphere.color * Lighting(P, N, -d, closest_sphere.specular)

    # предел рекурсии или не отражающий объект 
    #r = closest_sphere.reflective
    #if depth <= 0 or r <= 0:
    #    return local_color

    # вычисление отражённого цвета
    #R = ReflectRay(-d, N)
    #reflected_color = TraceRay(P, R, 0.001, inf, depth - 1)
    return local_color#*(1 - r) + reflected_color*r

In [53]:
class light():
    def __init__(self, t, i, d = np.array([0, 0, 0])):
        self.typelight = t
        self.intensity = i
        self.direction = d
        
lights = []
lights.append(light('ambient', 0.2))
#lights.append(light('point', 0.8, np.array([0, 3, 0])))
#lights.append(light('point', 0.17, np.array([2.1, 1.9, 3.9])))
#lights.append(light('point', 0.17, np.array([1.9, 1.9, 3.9])))
#lights.append(light('point', 0.17, np.array([2.1, 2.1, 3.9])))
#lights.append(light('point', 0.17, np.array([1.9, 2.1, 3.9])))
#lights.append(light('point', 0.17, np.array([2.1, 1.9, 4.1])))
#lights.append(light('point', 0.17, np.array([1.9, 1.9, 4.1])))
#lights.append(light('point', 0.17, np.array([2.1, 2.1, 4.1])))
lights.append(light('point', 0.8, np.array([1.5, 1.5, 4.2])))

def Lighting(P, N, V, s):
    i = 0.0
    for light in lights:
        if light.typelight == 'ambient':
            i += light.intensity
        else:
            if light.typelight == 'point':
                L = light.direction - P
                tmax = sqrt(np.sum(np.square(L)))
                
            # Проверка тени
            shadow_sphere, shadow_t = ClosestIntersection(P, L, 0.001, tmax)
            
            if shadow_sphere == None or shadow_sphere.spheretype == 'spherelight': 
                # матовые
                n_dot_l = np.sum(np.dot(N, L))
                if n_dot_l > 0:
                    i += light.intensity*n_dot_l/(sqrt(np.sum(np.square(N))) * sqrt(np.sum(np.square(L))))

                # зеркальные
                if s != -1:
                    R = 2*N*np.sum(np.dot(N, L)) - L
                    r_dot_v = np.sum(np.dot(R, V))
                    if r_dot_v > 0:
                        i += light.intensity * pow(r_dot_v/(sqrt(np.sum(np.square(R)))*sqrt(np.sum(np.square(V)))), s)
    return i

def ReflectRay(R, N):
    return 2*N*np.sum(np.dot(N, R)) - R;

In [54]:
O = np.array([0,0,0])
recursion_depth = 1

image = Image.open(r'canvas.png')

ImageW, ImageH = image.size
dist = 10
WindowW = 10
WindowH = 10
BACKGROUND_COLOR = np.array([0, 0, 0])

def CanvasToView(x,y):
    return np.array([x*WindowW/ImageW, -y*WindowH/ImageH, dist])

for x in tqdm(range(-300, 300)):
    xx = x + 300
    for y in range(-300, 300):
        yy = y + 300
        color = TraceRay(O, CanvasToView(x, y), 0.001, inf, recursion_depth)
        image.putpixel((xx, yy), tuple(int(c) for c in color))
image.show()

100%|██████████| 600/600 [02:18<00:00,  4.32it/s]
